In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import os

In [ ]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Certificate issues: all of a sudden, we started having certificate issues with our scraping requests. I'm going to blame Ryan for his amazing 30+ hr scraping fest. :)

Here's what we're trying to circumvent those issues: https://levelup.gitconnected.com/solve-the-dreadful-certificate-issues-in-python-requests-module-2020d922c72f

## Filtering
Here are the filters that are going to help us reduce each file before we concatenate it onto the master DWI misdemeanor or felany file. The following is from Marina, and she says to remember that the search terms are partial phrases so use `contains` when you're filtering by description.

### Words/phrases to search for offense descriptions: 

****misdemeanors*****

- DUI
- DWI
- DRV
- DRIV
- Driving while intox
- Obstruct highway
- Obstruct passageway
 
****felonies*****
 
- DWI
- Driving while intoxicated
- Driv 
- Intox
 
### Offense codes
 
****misdemeanors***** 
 
- 530720 -- obstruct passageway/roadway
- 530721 -- obstruct highway-intoxication
 
- 540400 -- driving while intoxicated
- 540401 -- driving while intoxicated 
- 540403 -- driving while intoxicated 1st
- 540404 -- driving while intoxicated 2nd 
- 540406 -- driving while intoxicated SB 
- 540408 -- drv while intox-open container
- 540409 -- driving while intoxicated 
- 540410 -- driving while intox-open can 
- 540411 -- driving while intoxicated-2d
- 540412 -- driving while intoxicated-2d/m and 3d/m
- 540416 -- dwi w/bac 0.15 or higher
- 540431 -- DUI - alcohol - minor
- 540510 -- DWLI - enhanced   and DWLS - enhanced 

****felonies***** 
 
- 90901 -- intoxication manslaughter
- 90902 -- intox manslaughter - publ serv
- 139901 -- intoxication assault
- 139902 -- intox assault-p/s - sbi 
- 540400 --driving while intoxicated 
- 540401 -- driving while intoxicated 
- 540402 -- driving while intoxicated
- 540403 -- driving while intoxicated 1st 
- 540405 -- driving while intoxicated 3rd -- driving while intoxicated 3r
- 540411 -- driving while intoxicated-2d
- 540412 -- driving while intoxicated-3d/m 
- 540413 -- intoxication assault 
- 540414 -- driv while intox-w/child < 15

She seems to suggest that we filter two different ways and get a combination of records that are in either one or both of those filtered datasets.

But you probably should do some routine cleaning too:
- make everything all caps or all lower
- remove double and maybe triple spaces

## MISDEMEANORS

### Download all the misdemeanor data

In [ ]:
url = 'https://www.bexar.org/2923/Misdemeanor-Records'
req = requests.get(url, headers, verify='./bexar-consolidate.pem')

soup = BeautifulSoup(req.content, 'html.parser')
docs_list = soup.find(class_='id6073f697-8a50-445c-a25a-c750e3d83b77')

for doc_link in docs_list.find_all('a'):
    
    download_url = doc_link.get('href')
    outpath = '../data/misdemeanors/'+ download_url.replace('http://edocs.bexar.org/cc/','')
    print(download_url)

    x = requests.get(download_url, verify = False)
    url_content = x.content

    csv_file = open(outpath, 'wb')

    csv_file.write(url_content)
    csv_file.close()

### Pull DWI records out of the misdemeanor data from our downloaded files

In [ ]:
mis_desc_list = ['dui','dwi','drv','driv','driving while intox',
                 'obstruct highway','obstruct passageway', 'intox', 'intoxication', 'alcohol']

misc_code_list = [90901, 90902, 139901, 139902, 139903, 530720, 530721, 540403, 540409, 540410, 540411, 540412, 540413, 
540414, 540415, 540416, 540417, 540418, 540421, 540423, 540430, 540431, 540432, 540433, 540434, 540520]

outfile = '../data/misdemeanors/bexar-misds-DWI-20191001-20191130.csv'

basepath = '../data/misdemeanors/'

misdemeanor_dfs = []
for filename in os.listdir(basepath):
    filepath = os.path.join(basepath, filename)
    
    if filename.endswith('.csv'):
        print('Working on ' + filename)
        df = pd.read_csv(filepath, error_bad_lines=False)

        #add filtering cause this shit is too big

        #filter by OFFENSE-CODE and OFFENSE-DESC
        df['offense_desc_clean'] = df['OFFENSE-DESC'].str.lower().str.replace('\s+', ' ', regex=True)
        df = df.loc[(df['offense_desc_clean'].str.contains('|'.join(mis_desc_list))) | (df['OFFENSE-CODE'].isin(misc_code_list))]
        #df = df.loc[(df['OFFENSE-CODE'].isin(misc_code_list))|(df['offense_desc_clean'].str.contains('|'.join(mis_desc_list))) ]
        #df = df.loc[(df['OFFENSE-CODE'].astype(str).str.contains('|'.join(misc_code_list)))]

        #filter by OFFENSE-DATE
        df['offense_date_clean'] = pd.to_datetime(df['OFFENSE-DATE'],infer_datetime_format=True)
        df = df.loc[df['offense_date_clean']>'2009-01-01']

        print('Records kept: ',len(df))
        #display(df.head())
        df.to_csv('../data/misdemeanors/refined/'+filename)
        misdemeanor_dfs.append(df)

misds = pd.concat(misdemeanor_dfs)

print(len(df))

#print to csv
misds.to_csv(outfile)

## FELONIES

### Download all the felony data

The following is a version of the felony records scraper script I wrote for Emilie Easton. This scrapes sexual assault cases, not DWI cases. You'll want to comment out the DWI descriptions and codes before you run for that project. 

In [ ]:
url = 'https://www.bexar.org/2988/Online-District-Clerk-Criminal-Records'
req = requests.get(url, headers, verify='./bexar-consolidate.pem')

soup = BeautifulSoup(req.content, 'html.parser')
docs_list = soup.find(id='graphicLinkWidget1fb208c8-8ef9-4403-b98a-e773308f98d0')

for doc_link in docs_list.find_all('a'):
    
    download_url = doc_link.get('href')
    outpath = '../data/felony/'+ download_url.replace('https://edocs.bexar.org/cc/','')
    print(download_url)

    x = requests.get(download_url, verify = False)
    url_content = x.content

    csv_file = open(outpath, 'wb')

    csv_file.write(url_content)
    csv_file.close()

### Pull DWI records out of the felony data from our downloaded files

In [ ]:
#felony records: https://www.bexar.org/2988/Online-District-Clerk-Criminal-Records
#get all divs with class widgetItem
#list href from first a element within widgetItem element
#add to list so we can bulk download with a single script

#the file you want to create from your scraped, filtered records
outfile = '../data/felony/bexar-felony-dwi-20191001-20191130.csv'

# CAUTION, these are sexual assault codes and descriptions. Comment these out
# and activate the DWI codes and descriptions below in order to use for DWI project
#fel_desc_list = ['sex assault','sexual assault']
#fel_code_list = [110010, 110011, 110012, 110020, 110021, 110041, 360108,
                   # 360111, 360112, 360114, 360502, 361250, 361251, 361252]

# THESE ARE THE DWI CODES/DESCRIPTIONS
fel_desc_list = ['dwi','driving while intoxicated','driv','intox']
fel_code_list = [90901, 90902, 139901, 139902, 139903, 530720, 530721, 540403, 540409, 540410, 540411, 540412, 540413, 
540414, 540415, 540416, 540417, 540418, 540421, 540423, 540430, 540431, 540432, 540433, 540434, 540520]

basepath = '../data/felony/'

felony_dfs = []
for filename in os.listdir(basepath):
    filepath = os.path.join(basepath, filename)
    
    if filename.endswith('.csv'):
        print('Working on ' + filename)
        df = pd.read_csv(filepath)

        #filter by OFFENSE-CODE and OFFENSE-DESC
        df['offense_desc_clean'] = df['OFFENSE-DESC'].str.lower().str.replace('\s+', ' ', regex=True)
        df = df.loc[(df['offense_desc_clean'].str.contains('|'.join(fel_desc_list))) | (df['OFFENSE-CODE'].isin(fel_code_list))]


        #filter by OFFENSE-DATE
        df['offense_date_clean'] = pd.to_datetime(df['OFFENSE-DATE'],infer_datetime_format=True)
        df = df.loc[df['offense_date_clean']>'2009-01-01']

        print('Records kept: ',len(df))
        df.to_csv('../data/felony/refined/'+filename)
        felony_dfs.append(df)

fels = pd.concat(felony_dfs)

#print to csv
fels.to_csv(outfile)

## ANALYSIS

### General shit to remember

Get a count on blanks for all these columns that you're grouping by. So make sure you know the universe before you get the stat.

In [ ]:
misds[['last_name','first_name']] = misds['FULL-NAME'].str.rsplit(',',n=1,expand=True)
misds['offense_year'] = pd.DatetimeIndex(misds['OFFENSE-DATE']).year

In [ ]:
print(len(misds))
print(misds.offense_date_clean.max())
print(misds.offense_date_clean.min())
display(misds.tail())

### Sorted by top offense amounts

In [ ]:
misds.sort_values('FINE-AMOUNT',ascending=False).head()[['']]

In [ ]:
misds.loc[(misds['SID'] == 825681)]

### Grouped by race

Marina is going to go down to the courthouse and ask the people there what all of the race codes mean.

In [ ]:
misds.groupby('RACE')['CASE-CAUSE-NBR'].count()

### Grouped by gender

In [ ]:
misds.groupby('SEX')['CASE-CAUSE-NBR'].count()

In [ ]:
print('Female:',19086/78133)
print('Male:',59047/78133)

### Group by offense type

In [ ]:
by_type = misds.groupby('offense_desc_clean')['CASE-CAUSE-NBR'].count().reset_index().sort_values('CASE-CAUSE-NBR',ascending=False)
by_type['pct'] = (by_type['CASE-CAUSE-NBR']/78133)*100

by_type#.to_csv('../data/offense-types-to-filter.csv')

### Grouped by offense year

In [ ]:
misds.groupby('offense_year')['CASE-CAUSE-NBR'].count().reset_index().sort_values('CASE-CAUSE-NBR',ascending=False)

### Grouped by individual

Maybe better to use the SID instead or in addition to this name + BOD grouping. SID might just be for booked and arrested peeps. Can figure this out by running NULL analysis on SID column. If there are null values then we know that it's not the greatest to group by.

Let's pull out the number of individuals who have 4 and then 4+ DWI offenses (just DWI not obst of hwy). Then let's pull out the number of individuals who have 4 and then 4+ DWI+OOH and see how many more peeps should have been carried over to the felony side of things.

In [ ]:
misds.groupby(['last_name','first_name','SID','BIRTHDATE'])['CASE-CAUSE-NBR'].count().reset_index().sort_values('CASE-CAUSE-NBR',ascending=False).head(50)

### Group by ATTORNEY-APPOINTED-RETAINED, RACE

This could be an interesting insight into equity. Probably the peeps who can afford to hire attorneys rather than be appointed attorneys will get a better deal.

So let's look at race by attorney for sure.

### Group by disposition

Disposition is the outcome of the case

Let's group by disposition and reduced offense to see whether people plead down (reduction) to a lesser charge and then were convicted (disposition).

So groupby offense types and disposition. Of people who get first time DWIs (offense type), how many were convicted, dismissed, etc.

Also groupby offense type, disposition and reduction.

### Group by offense-type, race, bond-status and bond-amt

See if there are gaps between racial groups. Maybe also just group by race and bond-amt to see what's up there.

### Group by ATTORNEY-BAR-NBR

### Additional questions

How many people have gotten three, four or five or more DWI charges over a certain time period?

How does the case disposition differ between people who are represented by a privately-hired attorney vs. public offender?

How many people reoffend after pleading down to obstruction of highway-intoxication? 

How does the conviction rate differ by court?

How many people convicted of intoxication assault or intoxication manslaughter get charged with a DWI again? (I found a couple of these just poking around through the data) 

How many people convicted of intoxication assault/manslaughter were convicted/charged with a DWI previously?

### FELONY: Why are intox manslaughter cases so much lower than fatalities?

The number of intox manslaughter cases each year compared to total number of alcohol-related fatalities (TXDOT)

Allie you just do intox manslaughter cases by year. Marina has alcohol-related fatalities data.